In [274]:
import pandas as pd

data = pd.read_csv('train.csv')


In [275]:
rows = data.loc[data['Cabin'].notna()]


In [276]:
Name = data['Name'].str.split(' ')
lastName = Name.str[0]
firstName = Name.str[2]
data = data.drop('Name', axis=1)
data['lastName'] = lastName
data['firstName'] = firstName


In [277]:
gender = {'male':0,'female':1}
port = {'C':0, 'Q':1, 'S':2}
data['Sex'] = data['Sex'].map(gender)
data['Embarked'] = data['Embarked'].map(port)
data['Age'] = data['Age'].fillna(data['Age'].mode()[0])
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,lastName,firstName
0,1,0,3,0,22.0,1,0,A/5 21171,7.2500,NaN,2.0,"Braund,",Owen
1,2,1,1,1,38.0,1,0,PC 17599,71.2833,C85,0.0,"Cumings,",John
2,3,1,3,1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2.0,"Heikkinen,",Laina
3,4,1,1,1,35.0,1,0,113803,53.1000,C123,2.0,"Futrelle,",Jacques
4,5,0,3,0,35.0,0,0,373450,8.0500,NaN,2.0,"Allen,",William
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.0,0,0,211536,13.0000,NaN,2.0,"Montvila,",Juozas
887,888,1,1,1,19.0,0,0,112053,30.0000,B42,2.0,"Graham,",Margaret
888,889,0,3,1,24.0,1,2,W./C. 6607,23.4500,NaN,2.0,"Johnston,",Catherine
889,890,1,1,0,26.0,0,0,111369,30.0000,C148,0.0,"Behr,",Karl


In [278]:
data = data.drop(['Ticket', 'Cabin', 'PassengerId'], axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,lastName,firstName
0,0,3,0,22.0,1,0,7.2500,2.0,"Braund,",Owen
1,1,1,1,38.0,1,0,71.2833,0.0,"Cumings,",John
2,1,3,1,26.0,0,0,7.9250,2.0,"Heikkinen,",Laina
3,1,1,1,35.0,1,0,53.1000,2.0,"Futrelle,",Jacques
4,0,3,0,35.0,0,0,8.0500,2.0,"Allen,",William
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,2.0,"Montvila,",Juozas
887,1,1,1,19.0,0,0,30.0000,2.0,"Graham,",Margaret
888,0,3,1,24.0,1,2,23.4500,2.0,"Johnston,",Catherine
889,1,1,0,26.0,0,0,30.0000,0.0,"Behr,",Karl


In [279]:
from sklearn.model_selection import cross_val_score
import numpy as np

def validate(X, y, clf):
    scores = []
    for i in ['neg_mean_squared_error', 'accuracy', 'roc_auc', 'f1', 'r2']:
        score = np.mean(cross_val_score(clf, X, y,scoring=i, cv=10))
        scores.append(f'{i}: {score}')
        print(f'{i}: {cross_val_score(clf, X, y,scoring=i, cv=10)}')
    return scores

In [280]:
from sklearn.model_selection import train_test_split
X = data.drop('Survived', axis=1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [284]:
import plotly.express as px
from sklearn.decomposition import PCA

def plotPCA(X, n_components):
    pca = PCA(n_components = n_components)
    components = pca.fit_transform(X)
    fig = px.scatter(components, x=0, y=1, color=data['Survived'])
    fig.show()

Tree Session:

Pure Tree

In [287]:
from sklearn import tree

def compareTree(X_train, y_train):
    results = []
    for i in range(3,5):
        for j in range(10,50, 10):
            clf = tree.DecisionTreeClassifier(max_depth=i, min_samples_split=j)
            clf = clf.fit(X_train, y_train)
            results.append([f'max depth {i} min samples {j}', validate(X_train, y_train, clf)])
    return results

In [288]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=3, min_samples_split=30)
clf = clf.fit(X_train, y_train)
validate(X_train, y_train, clf)


import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, feature_names = X.columns, class_names = ['Died', 'Survived']) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 


ValueError: could not convert string to float: 'Partner,'

In [ ]:
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predictions)


Trying feature engineering

In [ ]:
# familyStatus = 1 -> with family
# familyStatus = 0 -> with no family
arr1 = np.where(data['SibSp'] > 0, 1, 0)
arr2 = np.where(data['Parch'] > 0, 1, 0)
data['familyStatus'] = np.where((arr1 + arr2) > 0, 1, 0)
data

In [ ]:
# trying to get first and last names to see if it makes a difference

Name = data['Name'].str.split(' ')
lastName = Name.str[0]
firstName = Name.str[2]
data = data.drop('Name', axis=1)
data['lastName'] = lastName
data['firstName'] = firstName


In [ ]:
data = data.drop('SibSp', axis = 1)
data = data.drop('Parch', axis = 1)
data = data.drop('Survived', axis=1)
#X = data.drop('Survived', axis=1)


In [ ]:
from sklearn import tree


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = tree.DecisionTreeClassifier(max_depth=3, min_samples_split=30)
clf = clf.fit(X_train, y_train)
validate(X_train, y_train, clf)


import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, feature_names = X.columns, class_names = ['Died', 'Survived']) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 


FamilyStatus made no difference 

making a pipeline to send results:

In [ ]:
def pipeline(data):
    data = data.drop('Name', axis=1)    
    gender = {'male':0,'female':1}
    port = {'C':0, 'Q':1, 'S':2}
    data['Sex'] = data['Sex'].map(gender)
    data['Embarked'] = data['Embarked'].map(port)
    data['Age'] = data['Age'].fillna(data['Age'].mode()[0])
    data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
    data = data.drop(['Ticket', 'Cabin'], axis=1)
    data['Fare'] = data['Fare'].fillna(data['Fare'].mode()[0])

    return data


In [ ]:
dataTest = pd.read_csv('test.csv')
X_test = pipeline(dataTest)
predictions = clf.predict(X_test)
df = pd.DataFrame({'PassengerId': X_test['PassengerId'], 'Survived': predictions})
df.to_csv('predictions.csv', index=False)